In [19]:
from graphviz import Digraph

class Node:
    """Класс для узла дерева"""
    def __init__(self, value):
        self.value = value  # Значение узла (оператор или операнд)
        self.left = None    # Левый дочерний узел
        self.right = None   # Правый дочерний узел


def build_expression_tree(expression):
    """Построение дерева выражения из строки"""
    stack = []  # Стек для хранения узлов
    operators = set("+-*/^")  # Набор допустимых операторов
    postfix = convert_to_postfix(expression)  # Преобразуем в постфиксную запись

    for char in postfix:
        if char.isalnum():  # Если это операнд (a-z, 0-9)
            stack.append(Node(char))
        elif char in operators:  # Если это оператор
            node = Node(char)
            node.right = stack.pop()  # Правый операнд
            node.left = stack.pop()   # Левый операнд
            stack.append(node)

    return stack[0]  # Корень дерева


def inorder_traversal(node, parent_precedence=0, is_right_child=False):
    """
    Скобки добавляются только там, где это необходимо, включая одинаковый приоритет.
    """
    if node is None:
        return ""

    # Операнды (листья дерева) возвращаются без скобок
    if node.left is None and node.right is None:
        return node.value

    # Устанавливаем приоритеты для операций
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}
    current_precedence = precedence.get(node.value, 0)

    # Рекурсивно обходим левое и правое поддерево
    left = inorder_traversal(node.left, current_precedence)
    right = inorder_traversal(node.right, current_precedence, is_right_child=True)

    # Формируем строку текущего узла
    result = f"{left}{node.value}{right}"

    # Добавляем скобки, если:
    # 1. Текущий оператор имеет меньший приоритет, чем родительский.
    # 2. Это правый дочерний узел, и текущий оператор имеет равный приоритет с родительским (нужно сохранить порядок).
    if current_precedence < parent_precedence or (is_right_child and current_precedence == parent_precedence):
        result = f"({result})"

    return result



def visualize_tree(node, graph=None, parent=None):
    """Визуализация дерева с помощью Graphviz"""
    if graph is None:
        graph = Digraph(format='png')
        graph.attr(dpi='300')  # Устанавливаем высокое разрешение

    node_id = str(id(node))  # Уникальный идентификатор узла
    graph.node(node_id, label=node.value)  # Добавляем узел в граф

    if parent is not None:
        graph.edge(parent, node_id)  # Добавляем связь между узлами

    if node.left:
        visualize_tree(node.left, graph, node_id)  # Рекурсивно строим левое поддерево
    if node.right:
        visualize_tree(node.right, graph, node_id)  # Рекурсивно строим правое поддерево

    return graph


def main():
    expression = "(a+b)*(c*(d+e+f))+g*(i+j^2)"  # Исходное выражение
    print(f"Исходное выражение: {expression}")

    # Шаг 1: Построить дерево
    root = build_expression_tree(expression)
    print("Дерево выражений построено.")

    # Шаг 2: Проверить корректность дерева
    result = inorder_traversal(root)
    print(f"Результат обратного перехода к выражению: {result}")

    # Проверяем совпадение с исходным выражением (после удаления лишних скобок)
    if result == expression:
        print("Дерево выражений корректно.")
    else:
        print("Ошибка в дереве выражений!")

    # Шаг 3: Построение графического дерева
    print("Создаём графическое дерево...")
    graph = visualize_tree(root)
    graph.render('expression_tree', view=True)  # Сохраняем и открываем файл


if __name__ == "__main__":
    main()


Исходное выражение: (a+b)*(c*(d+e+f))+g*(i+j^2)
Дерево выражений построено.
Результат обратного перехода к выражению: (a+b)*(c*(d+e+f))+g*(i+j^2)
Дерево выражений корректно.
Создаём графическое дерево...
